In [1]:
!pip install -q langchain

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi3:instruct", timeout=120.0, base_url="http://host.docker.internal:11434", num_gpu=1)

In [2]:
from operator import itemgetter
import requests
from typing import Dict, List

from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool
from langchain_core.output_parsers import JsonOutputParser

In [3]:
HOST = "api.frankfurter.app"

@tool
def convert_currency(amount: float, currency1: str, currency2: str) -> dict:
    """Converts currency."""
    url = f"https://{HOST}/latest?amount={amount}&from={currency1.upper()}&to={currency2.upper()}"
    resp = requests.get(url=url)
    
    return resp.json()
    
tools = [convert_currency]


def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [4]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([convert_currency])
rendered_tools

'convert_currency: convert_currency(amount: float, currency1: str, currency2: str) -> dict - Converts currency.'

In [30]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """[INST] <<SYS>>You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

<tools>
{tools}
</tools>

If you don't know the answer, just say that you don't know. Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys. Format the inputs properly as described by the tool.

<</SYS>>

 {question} [/INST]"""


prompt = ChatPromptTemplate.from_template(system_prompt)

chain = prompt | llm | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)

In [31]:
prompt

ChatPromptTemplate(input_variables=['question', 'tools'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'tools'], template="[INST] <<SYS>>You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\n<tools>\n{tools}\n</tools>\n\nIf you don't know the answer, just say that you don't know. Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys. Format the inputs properly as described by the tool.\n\n<</SYS>>\n\n {question} [/INST]"))])

In [20]:
chain.invoke({"tools": rendered_tools, "question": "convert 10 British Pounds to USD"})

{'name': 'convert_currency',
 'arguments': {'amount': 10, 'currency1': 'GBP', 'currency2': 'USD'},
 'output': {'amount': 10.0,
  'base': 'GBP',
  'date': '2024-04-30',
  'rates': {'USD': 12.5389}}}

# Agent 

In [9]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="phi3:instruct", timeout=120.0, base_url="http://host.docker.internal:11434", num_gpu=1, temperature=0)

In [10]:
from operator import itemgetter
import requests
from typing import Dict, List, Optional, Type

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

HOST = "api.frankfurter.app"


def convert_currency(amount: float, currency1: str, currency2: str) -> str:
    """Converts currency.
        amount (float): The amount of currency to convert. It must be always numeric.
        currency1 (str): The currency to convert from. It must be a valid currency code. Examples: USD, EUR, GBP.
        currency2 (str): The currency to convert to. It must be a valid currency code. Examples: USD, EUR, GBP.
    """
    try:
        url = f"https://{HOST}/latest?amount={amount}&from={currency1.upper()}&to={currency2.upper()}"
        resp = requests.get(url=url)
    
        return resp.text
    except Exception as e:
        return str(e)
    
class ConvertCurrencyInput(BaseModel):
    amount: float = Field(..., title="Amount", description="The amount of currency to convert. It must be always numeric.")
    currency1: str = Field(..., title="Currency 1", description="The currency to convert from. It must be a valid currency code. Examples: USD, EUR, GBP.")
    currency2: str = Field(..., title="Currency 2", description="The currency to convert to. It must be a valid currency code. Examples: USD, EUR, GBP.")
    
    
class ConvertCurrencyTool(BaseTool):
    name = "convert_currency"
    description = "Usefull for converting currency."
    args_schema: Type[BaseModel] = ConvertCurrencyInput
    return_direct: bool = True
    
    def _run(
            self, 
            amount: float, 
            currency1: str, 
            currency2: str, 
            run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Tool to convert currency."""
        return convert_currency(amount, currency1, currency2)
    
    async def _run_async(
            self, 
            amount: float, 
            currency1: str, 
            currency2: str, 
            run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Tool to convert currency."""
        return convert_currency(amount, currency1, currency2)
    


In [34]:
tools = [ConvertCurrencyTool()]

In [35]:
tools

[ConvertCurrencyTool()]

In [36]:
from langchain.tools.render import render_text_description_and_args
rendered_tools = render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')
tool_names = [t.name for t in tools]

In [37]:
rendered_tools

"convert_currency: Usefull for converting currency., args: {{'amount': {{'title': 'Amount', 'description': 'The amount of currency to convert. It must be always numeric.', 'type': 'number'}}, 'currency1': {{'title': 'Currency 1', 'description': 'The currency to convert from. It must be a valid currency code. Examples: USD, EUR, GBP.', 'type': 'string'}}, 'currency2': {{'title': 'Currency 2', 'description': 'The currency to convert to. It must be a valid currency code. Examples: USD, EUR, GBP.', 'type': 'string'}}}}"

In [40]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", f"""Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{rendered_tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""),
    ("placeholder", "{chat_history}"),
    ("human", """{input}

{agent_scratchpad}
 (reminder to respond in a JSON blob no matter what)"""),
])

In [41]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} partial_variables={'chat_history': []} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nconvert_currency: Usefull for converting currency., args: {{\'amount\': {{\'title\': \'Amount\', \'description\': \'The amount of currency to convert. It must be always numeric.\', \'type\': \'number\'}}, \'currency1\': {{\'title\': \'Currency 1\', \'description\': \'The currency to convert from. It must be a valid currency code. Examples: USD, EUR, GBP.\', \'type\': \'string\'}}, \'currency2\': {{\

In [42]:
from typing import Tuple
from langchain_core.messages import AIMessage, HumanMessage

def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [43]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_messages
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser, 
    JSONAgentOutputParser,
)


chat_model_with_stop = llm.bind(stop=["\nObservation"])


agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_messages(x["intermediate_steps"]),
        "chat_history": lambda x: (
            _format_chat_history(x["chat_history"]) if x.get("chat_history") else []
        ),
    }
    | prompt
    | chat_model_with_stop
    | JSONAgentOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )
    
    
executor = (AgentExecutor(
    agent=agent, 
    tools=tools, 
    return_intermediate_steps=True,
    verbose=True,
    handle_parsing_errors=True,
    ).with_types(input_type=AgentInput))

In [44]:
executor.invoke({"input": "convert 1 British Pounds to USD", "chat_history": []})



> Entering new AgentExecutor chain...
{
  "action": "convert_currency",
  "action_input": {
    "amount": 1,
    "currency1": "GBP",
    "currency2": "USD"
  }
}{"amount":1.0,"base":"GBP","date":"2024-04-30","rates":{"USD":1.2539}}


> Finished chain.


{'input': 'convert 1 British Pounds to USD',
 'chat_history': [],
 'output': '{"amount":1.0,"base":"GBP","date":"2024-04-30","rates":{"USD":1.2539}}',
 'intermediate_steps': [(AgentAction(tool='convert_currency', tool_input={'amount': 1, 'currency1': 'GBP', 'currency2': 'USD'}, log='{\n  "action": "convert_currency",\n  "action_input": {\n    "amount": 1,\n    "currency1": "GBP",\n    "currency2": "USD"\n  }\n}'),
   '{"amount":1.0,"base":"GBP","date":"2024-04-30","rates":{"USD":1.2539}}')]}

In [49]:
executor.invoke({"input": "convert 1 euros to Brazilian Real"})



> Entering new AgentExecutor chain...
{
  "action": "convert_currency",
  "action_input": {
    "amount": 1,
    "currency1": "EUR",
    "currency2": "BRL"
  }
}{"amount":1.0,"base":"EUR","date":"2024-04-30","rates":{"BRL":5.4928}}


> Finished chain.


{'input': 'convert 1 euros to Brazilian Real',
 'output': '{"amount":1.0,"base":"EUR","date":"2024-04-30","rates":{"BRL":5.4928}}',
 'intermediate_steps': [(AgentAction(tool='convert_currency', tool_input={'amount': 1, 'currency1': 'EUR', 'currency2': 'BRL'}, log='{\n  "action": "convert_currency",\n  "action_input": {\n    "amount": 1,\n    "currency1": "EUR",\n    "currency2": "BRL"\n  }\n}'),
   '{"amount":1.0,"base":"EUR","date":"2024-04-30","rates":{"BRL":5.4928}}')]}

In [46]:
executor.invoke({"input": "convert 10 British Pounds to USD", "chat_history": []})



> Entering new AgentExecutor chain...
{
  "action": "convert_currency",
  "action_input": {
    "amount": 10,
    "currency1": "GBP",
    "currency2": "USD"
  }
}{"amount":10.0,"base":"GBP","date":"2024-04-30","rates":{"USD":12.5389}}


> Finished chain.


{'input': 'convert 10 British Pounds to USD',
 'chat_history': [],
 'output': '{"amount":10.0,"base":"GBP","date":"2024-04-30","rates":{"USD":12.5389}}',
 'intermediate_steps': [(AgentAction(tool='convert_currency', tool_input={'amount': 10, 'currency1': 'GBP', 'currency2': 'USD'}, log='{\n  "action": "convert_currency",\n  "action_input": {\n    "amount": 10,\n    "currency1": "GBP",\n    "currency2": "USD"\n  }\n}'),
   '{"amount":10.0,"base":"GBP","date":"2024-04-30","rates":{"USD":12.5389}}')]}

In [47]:
executor.invoke({"input": "convert 1 British Pounds to Brazilian Real", "chat_history": []})



> Entering new AgentExecutor chain...
{
  "action": "convert_currency",
  "action_input": {
    "amount": 1,
    "currency1": "GBP",
    "currency2": "BRL"
  }
}{"amount":1.0,"base":"GBP","date":"2024-04-30","rates":{"BRL":6.426}}


> Finished chain.


{'input': 'convert 1 British Pounds to Brazilian Real',
 'chat_history': [],
 'output': '{"amount":1.0,"base":"GBP","date":"2024-04-30","rates":{"BRL":6.426}}',
 'intermediate_steps': [(AgentAction(tool='convert_currency', tool_input={'amount': 1, 'currency1': 'GBP', 'currency2': 'BRL'}, log='{\n  "action": "convert_currency",\n  "action_input": {\n    "amount": 1,\n    "currency1": "GBP",\n    "currency2": "BRL"\n  }\n}'),
   '{"amount":1.0,"base":"GBP","date":"2024-04-30","rates":{"BRL":6.426}}')]}

In [5]:
import requests

data = {"text": "convert 1 British Pounds to USD", "chat_history": []}
response = requests.post("http://localhost:8000/chat", json=data)

In [6]:
response.json()

{'input': 'convert 1 British Pounds to USD',
 'output': '{"amount":1.0,"base":"GBP","date":"2024-05-02","rates":{"USD":1.2507}}',
 'intermediate_steps': ['(AgentAction(tool=\'convert_currency\', tool_input={\'amount\': 1, \'currency1\': \'GBP\', \'currency2\': \'USD\'}, log=\'{\\n  "action": "convert_currency",\\n  "action_input": {\\n    "amount": 1,\\n    "currency1": "GBP",\\n    "currency2": "USD"\\n  }\\n}\'), \'{"amount":1.0,"base":"GBP","date":"2024-05-02","rates":{"USD":1.2507}}\')']}